In [3]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownTextSplitter
from langchain.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import tqdm
import os

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [4]:
from langchain.document_loaders import PyPDFLoader
#from langchain_community.document_loaders import WebBaseLoader

loader = PyPDFLoader("docs/Sách Deep Learning cơ bản.pdf")
# loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

pages = loader.load()

In [ ]:
print(len(pages))

227


In [3]:

text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

In [4]:
docs = text_splitter.split_documents(pages)

In [5]:
model = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceEmbeddings(model_name = model)

d:\thực tập doanh nghiệp\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\thực tập doanh nghiệp\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\thực tập doanh nghiệp\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, u

In [ ]:
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_index")


In [6]:
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [8]:
question = "loss fuction"
docs = vectorstore.max_marginal_relevance_search(question,k=3)
docs[0].page_content

'130 Chương 9. Giới thiệu keras và bài toán phân loại ảnh\nĐể ý là label của data là số i là vector v kích thước 10*1 với vi+1=1và các giá trị khác bằng 0. So\nvới quy ước về phần trăm ở trên thì one-hot encoding có ý nghĩa là ta chắc chắn 100% ảnh này là số 5.\nGiờ ta có giá trị thật (label) dạng one-hot encoding giá trị dự đoán ở output layer sau hàm softmax\nfunction cùng kích thước 10*1. Ta cần định nghĩa hàm loss function để đánh giá độ tốt của model.\nMong muốn là a6gần 1 còn các giá trị a khác gần 0 vì như thế nghĩa là model dự đoán đúng được\nảnh đầu vào là ảnh số 5. Ta định nghĩa loss function:\nL=−10\n∑\ni=1yi∗log(ˆyi)'

In [7]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [8]:
llm = Ollama(model="llama3")

In [9]:
prompt_text = """
Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi.
Hãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh
Hãy trả lời không biết nêu không có thông tin tron đoạn ngữ cảnh

đoạn ngữ cảnh: {context}

câu hỏi: {question}

câu trả lời:
"""

prompt = ChatPromptTemplate.from_template(prompt_text)
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nBạn là trợ lý cho các nhiệm vụ trả lời câu hỏi.\nHãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh\nHãy trả lời không biết nêu không có thông tin tron đoạn ngữ cảnh\n\nđoạn ngữ cảnh: {context}\n\ncâu hỏi: {question}\n\ncâu trả lời:\n'))]


In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
question = "CNN là gì?"
rag_chain.invoke(question)

'CNN (Convolutional Neural Network) is a type of artificial neural network that is primarily used for image and signal processing. The main characteristic of a CNN is the use of convolutional layers, which apply filters to small regions in an image, and pooling layers, which downsample the output of each filter.\n\nIn essence, CNNs are designed to take advantage of the spatial hierarchies present in images by using convolutional and pooling layers to extract features at multiple scales. This allows them to learn invariant representations of objects that can be used for tasks such as object detection, image classification, and segmentation.\n\nCNNs have been widely used in many applications, including computer vision, natural language processing, and audio processing.'

In [15]:
def generate_response(input_text):
    response = rag_chain.invoke(input_text)
    return response

In [16]:
import gradio as gr


iface = gr.Interface(generate_response, 
                     inputs="textbox",
                     outputs="textbox",
                     title="Chatbot about Deep learning",
                     description="Enter your message and the chatbot will respond accordingly.")
iface.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
